In [1]:
import numpy as np
import pandas as pd
from collections import OrderedDict, defaultdict
import glob
import sys 
sys.path.append('..')
sys.path.append('../../../../utils')
sys.path.append('../../../../third')
import gezi
from gezi import tqdm

In [2]:
d = pd.read_csv('../input/feed_info.csv')

In [3]:
MAX_TAGS = 6
MAX_KEYS = 7
DESC_LEN = 64
DESC_CHAR_LEN = 128
WORD_LEN = 128
START_ID = 3
UNK_ID = 1
EMPTY_ID = 2
NAN_ID = -1

In [4]:
d.description.apply(lambda x: len(str(x).split())).describe([.90,.95,.99,.999])

count    106444.000000
mean         28.658853
std          22.637732
min           1.000000
50%          24.000000
90%          53.000000
95%          65.000000
99%         101.000000
99.9%       219.557000
max         622.000000
Name: description, dtype: float64

In [5]:
d = d.fillna(NAN_ID)

In [6]:
d.head(10)

,feedid,authorid,videoplayseconds,description,ocr,asr,bgm_song_id,bgm_singer_id,manual_keyword_list,machine_keyword_list,manual_tag_list,machine_tag_list,description_char,ocr_char,asr_char
0,43549,6165,38,104741 122649 8109 117252 65632 23463 118668 4...,139499 59421 82007 142955 27736 83577 52394 11...,142955 27736 83577 103956 32010 34170 89740 90...,19356.0,11703.0,15506;7715;17582,26334;219;25209;7715;18541,81;269;159;6,269 0.8525666;81 0.8525666;8 1.1e-07;306 0.0;2...,26439 5247 6426 3827 1882 26018 20744 22204 30...,25926 8491 13394 2203 26439 6243 33054 16435 1...,2203 26439 6243 33054 16435 16307 17070 24908 ...
1,77432,9386,60,35753 27736 146603 73055 11794 101761 11794 81...,35753 146603 73055 11794 101761 67496 16933 52...,146739 14368 79290 79213 47366 8109 33194 1198...,-1.0,-1.0,8199;18322;4243,24078;19924,194;267;159;6,267 0.99293476;194 0.99293476,31010 32495 6243 13923 15360 30483 2709 26084 ...,31010 32495 13923 15360 30483 2709 26084 15160...,7259 20851 5061 26207 17573 17531 15117 20072 ...
2,12921,2996,15,114413 107973 117252 27736 41035 32715 125374 ...,-1,-1,10288.0,11354.0,-1,11696;6926,202;23;160;6,23 0.64771646;202 0.64771646,27077 10050 27464 20072 26018 6243 21492 14750...,-1,-1
3,34538,12577,18,60473 9864 8109 135558 108803 89307 37630 6481...,39654 95201 22675 49521 135558 108803 89307 37...,7446 47082 37630 99660 9844 113002 9844 113002...,24435.0,13645.0,5454;1197;3727,-1,328;13;159;6,267 0.21298289;191 0.21298289;8 0.34298885;306...,21489 24672 2223 1882 4956 1545 10424 19791 12...,9015 21209 5026 11192 4956 1545 10424 19791 12...,20825 13395 14765 24513 7305 12837 10337 8885 ...
4,75414,7185,16,105860 4691 134820 55911 80449 79213 23233 139...,73055 66447 88970,104002 4438 104002 4438,23428.0,10730.0,17083;4123,8058;219;21639;15621;25209,81;269;159;6,269 0.86163938;81 0.86163938,25782 19583 7768 3907 14068 12681 5043 17859 1...,30483 23721 1518 22741 25118,23112 969 23112 969
5,34539,2595,43,123046 116187 8092 22091 77895 11717 93522 100...,61498 70163 1428 123046 116187 44971 3063 3214...,147566 45843 10952 25290 79030 11717 34198 993...,16541.0,17296.0,529;2666;26954,529;9298,45;239;159;169,239 0.68520623;45 0.68520623;8 0.0;306 0.0;207...,27442 1543 31605 1868 20857 22179 17181 7261 8...,13842 6439 32379 25274 8847 1251 27442 1543 31...,15371 6498 31376 30024 24307 21492 10083 20072...
6,101310,8115,15,106872 36623 26564 8109 110838 148383 26564 33...,9837 110838 148383 26564 33895 110690 133005,117252 44854 26564 110142 93536 91254 8109 983...,19367.0,10532.0,-1,18351;13433;25057;11002;14687;10664;19485;12865,9;6,237 0.90296459;246 0.90296459,20831 11704 11598 11354 5967 1882 1217 23805 1...,2200 1217 23805 10083 14040 5967 12194 13395 1...,26018 10155 5967 24513 20833 5311 32214 1882 2...
7,13769,11696,25,144240 54435 37630 117252 27736 40670 134820 1...,63733 106137 31785 73044 25794 65020 90843 107...,123075 11342 79301 23908 110100 112151 110677 ...,-1.0,-1.0,-1,8800;11061;15752,9;6,17 0.74826479;37 0.74826479,12783 22473 5277 10040 12837 26018 6243 8311 3...,20857 15640 7261 23039 30060 32499 5009 8481 7...,30191 2606 17583 5391 20861 15640 12836 4895 1...
8,6311,9533,60,102036 39661 66989 110142 118715 93522 24335 8...,122344 140404 55785 26564 93563 102383 27736 7...,90843 31279 117347 116237 8109 287 49575 56976...,-1.0,-1.0,-1,17892;18428,9;6,235 0.36773422;204 0.36773422,2682 32028 12649 7461 4913 29499 24513 24368 2...,12676 3901 31177 12653 5967 20857 3653 11877 6...,26018 2223 30031 2220 26054 18659 15360 1882 6...
9,6801,13501,14,45012 25146 45443 122452 875 80983 8109 25146 ...,20248 33368 37767 37767 82007 78435 69591 9354...,90290 77895 25146 106949 129539 106793 304 277...,-1.0,-1.0,15506;17582;10844,25209;219;4826;25500,81;269;159;6,269 0.96752864;81 0.96752864,10226 13505 15360 10326 17526 26208 253 12762 ...,27393 7536 7349 7349 17323 11893 20838 18755 3...,20085 17181 13505 15360 23786 14009 12831 2706...


In [9]:
# generate feed_info.npy

MIN_COUNT = None
# MIN_COUNT = 5
    
single_keys = ['author', 'song', 'singer']
multi_keys =  ['manual_tags', 'machine_tags', 'machine_tag_probs', 'machine_tags2', 'manual_keys', 'machine_keys', 'desc', 'desc_char', 'ocr', 'asr']
info_keys = single_keys + multi_keys
info_lens = [1] * len(single_keys) + [MAX_TAGS, MAX_TAGS, MAX_TAGS, MAX_TAGS, MAX_KEYS, MAX_KEYS, DESC_LEN, DESC_CHAR_LEN, WORD_LEN, WORD_LEN]

vocab_names = [
                'user', 'doc',
                'author', 'singer', 'song',
                'key', 'tag', 'word', 'char'
              ]
vocabs = {}

for vocab_name in vocab_names:
  vocab_file =  f'../input/{vocab_name}_vocab.txt'
  # if not doc then mask as UNK for rare words, tags, keys..
  min_count = None if vocab_name == 'doc' else MIN_COUNT
#   min_count = MIN_COUNT if vocab_name in ['word', 'char'] else None
  vocab = gezi.Vocab(vocab_file, min_count=min_count)
  vocabs[vocab_name] = vocab
    
d = pd.read_csv('../input/feed_info.csv')
d = d.fillna(NAN_ID)
d.feedid = d.feedid.astype(int)
d.authorid = d.authorid.astype(int)
d.bgm_singer_id = d.bgm_singer_id.astype(int)
d.bgm_song_id = d.bgm_song_id.astype(int)

def gen_unk():
  l = [0] * sum(info_lens)
  x = 0
  for len_ in info_lens:
    l[x] = UNK_ID
    x += len_
  return l

def gen_empty():
  l = [0] * sum(info_lens)
  x = 0
  for len_ in info_lens:
    l[x] = EMPTY_ID
    x += len_
  return l

embs = [
    gen_empty()
] * vocabs['doc'].size()
embs[0] = [0] * sum(info_lens)
# embs[1] = gen_unk()
embs[1] = [0] * sum(info_lens)
print(np.asarray(embs).shape)

def set_feature(feature, feed, key, feedid):
  key_ = key.replace('description', 'desc')
  if str(feed[key]) != str(NAN_ID):
    feature[key_] = list(map(int, str(feed[key]).split())) 
  else:
    feature[key_] = [UNK_ID]

#   MAX_LEN = 512 
  MAX_LEN = WORD_LEN
  if key_ == 'desc':
    MAX_LEN = DESC_LEN
  elif key_ == 'desc_char':
    MAX_LEN = DESC_CHAR_LEN
    
  vocab_name = 'char' if 'char' in key else 'word'
  vocab = vocabs[vocab_name]
  feature[key_] = [vocab.id(x) for x in feature[key_]]
  feature[key_] = gezi.pad(feature[key_], MAX_LEN, 0, last_tokens=10) 

GOOD_TAG_PROB = 0.2

for row in tqdm(d.itertuples(), total=len(d), desc='feed_info'):
  row = row._asdict()
  author = row['authorid']
  feedid = row['feedid']
  song = row['bgm_song_id']
  singer = row['bgm_singer_id']
  l = [
      vocabs['author'].id(author),
      vocabs['song'].id(song),
      vocabs['singer'].id(singer)
  ]
    
  manual_tags = str(row['manual_tag_list']).split(';')
  manual_tags_ = []
  for tag in manual_tags:
    if tag == 'nan' or tag == str(NAN_ID):
      continue
    else:
      manual_tags_.append(vocabs['tag'].id(int(tag)))
  if not manual_tags_:
    manual_tags_ = [UNK_ID]
  manual_tags_ = gezi.pad(manual_tags_, MAX_TAGS, 0)
  l += manual_tags_
  
  machine_tags = str(row['machine_tag_list']).split(';')
  machine_tags_ = []
  machine_tag_probs_ = []
  machine_tags2_ = []
  for item in machine_tags:
    if item == 'nan' or item == str(NAN_ID):
      continue
    else:
      x = item.split()
      tag, prob = int(x[0]), float(x[1])
      machine_tags_.append(vocabs['tag'].id(tag))
      machine_tag_probs_.append(int(prob * 1e7))
      if prob > GOOD_TAG_PROB:
        machine_tags2_.append(vocabs['tag'].id(tag))
  if not machine_tags_:
    machine_tags_ = [UNK_ID]
  machine_tags_ = gezi.pad(machine_tags_, MAX_TAGS, 0)
  l += machine_tags_
  machine_tag_probs_ = gezi.pad(machine_tag_probs_, MAX_TAGS, 0)
  l += machine_tag_probs_
  if not machine_tags2_:
    machine_tags2_ = [UNK_ID]
  machine_tags2_ = gezi.pad(machine_tags_, MAX_TAGS, 0)
  l += machine_tags2_
  

  manual_keys = str(row['manual_keyword_list']).split(';')
  manual_keys_ = []
  for key in manual_keys:
    if key == 'nan' or key == str(NAN_ID):
      continue
    else:
      manual_keys_.append(vocabs['key'].id(int(key)))
  if not manual_keys_:
    manual_keys_ = [UNK_ID]
  manual_keys_ = gezi.pad(manual_keys_, MAX_KEYS, 0)
  l += manual_keys_
    
  machine_keys = str(row['machine_keyword_list']).split(';')
  machine_keys_ = []
  for key in machine_keys:
    if key == 'nan' or key == str(NAN_ID):
      continue
    else:
      machine_keys_.append(vocabs['key'].id(int(key)))
  if not machine_keys_:
    machine_keys_ = [UNK_ID]
  machine_keys_ = gezi.pad(machine_keys_, MAX_KEYS, 0)
  l += machine_keys_
    
  feature = OrderedDict()
#   for key in ['description', 'description_char', 'ocr', 'ocr_char', 'asr', 'asr_char']:
  for key in ['description', 'description_char', 'ocr', 'asr']:
    set_feature(feature, row, key, feedid)
    
  for key in feature:
    l += feature[key]

  embs[vocabs['doc'].id(int(row['feedid']))] = l

print(np.asarray(embs).shape)

embs = np.asarray(embs)

print('--save to ../input/doc_lookup.npy')
np.save('../input/doc_lookup.npy', embs)

(106446, 489)


feed_info:   0%|          | 0/106444 [00:00<?, ?it/s]

(106446, 489)
--save to ../input/doc_lookup.npy


In [7]:
d.describe()

,feedid,authorid,videoplayseconds,bgm_song_id,bgm_singer_id
count,106444.000000,106444.000000,106444.000000,106444.000000,106444.000000
mean,56443.543704,9488.196639,34.446545,6306.111166,4416.853162
std,32582.304899,5384.927792,277.086122,8144.350397,5658.348052
min,0.000000,0.000000,2.000000,-1.000000,-1.000000
25%,28243.750000,4844.750000,14.000000,-1.000000,-1.000000
50%,56419.500000,9584.000000,26.000000,87.500000,78.000000
75%,84659.250000,14122.000000,54.000000,12592.000000,8663.250000
max,112871.000000,18788.000000,59960.000000,25158.000000,17499.000000


In [11]:
np.asarray(embs).shape

(106446, 489)

In [12]:
embs[vocabs['doc'].id(77432)]

array([   4141,       1,       1,      44,       8,       3,       2,
             0,       0,       8,      44,       0,       0,       0,
             0, 9929347, 9929347,       0,       0,       0,       0,
             8,      44,       0,       0,       0,       0,    8180,
            93,   11208,       0,       0,       0,       0,      67,
            57,       0,       0,       0,       0,       0,    2081,
             4,   14645,     277,     251,     229,     251,       2,
           377,    1134,     192,      16,      72,    2905,   14953,
             2,   10753,     629,     223,    1267,     108,    1293,
             7,     223,    2317,       0,       0,       0,       0,
             0,       0,       0,       0,       0,       0,       0,
             0,       0,       0,       0,       0,       0,       0,
             0,       0,       0,       0,       0,       0,       0,
             0,       0,       0,       0,       0,       0,       0,
             0,     

In [13]:
embs[vocabs['doc'].id(12912)]

array([   3191,   21281,     580,     139,      26,      31,       2,
             0,       0,      26,     139,       0,       0,       0,
             0, 4759715, 4759715,       0,       0,       0,       0,
            26,     139,       0,       0,       0,       0,       1,
             0,       0,       0,       0,       0,       0,    2414,
           977,    1179,       0,       0,       0,       0,    1243,
            33,    5449,   54195,      49,     365,     713,      20,
            86,       4,     826,       4,       5,       5,       5,
             5,       5,      50,       9,      24,     193,     240,
         21684,       5,      28,       7,    5366,       5,       7,
          1239,       5,       7,      33,    5449,   54195,       0,
             0,       0,       0,       0,       0,       0,       0,
             0,       0,       0,       0,       0,       0,       0,
             0,       0,       0,       0,       0,       0,       0,
             0,     

In [14]:
lens = [info_lens[0]] * len(info_lens)
for i in range(len(info_lens)):
  if i > 0:
    lens[i] = info_lens[i] + lens[i - 1]
embs_ = np.split(embs, lens, axis=-1)

In [15]:
sum(info_lens)

489

In [16]:

for key, len_, emb in zip(info_keys, info_lens, embs_):
  print(key, len_, emb.shape)
  np.save(f'../input/{key}_lookup.npy', emb)

author 1 (106446, 1)
song 1 (106446, 1)
singer 1 (106446, 1)
manual_tags 6 (106446, 6)
machine_tags 6 (106446, 6)
machine_tag_probs 6 (106446, 6)
machine_tags2 6 (106446, 6)
manual_keys 7 (106446, 7)
machine_keys 7 (106446, 7)
desc 64 (106446, 64)
desc_char 128 (106446, 128)
ocr 128 (106446, 128)
asr 128 (106446, 128)


In [17]:
np.load('../input/manual_keys_lookup.npy')[vocabs['doc'].id(12912)]

array([1, 0, 0, 0, 0, 0, 0])